In [1]:
import dgl
from dgl.data import DGLDataset
import torch
import os
import pandas as pd
import numpy as np
import torch
import itertools
import numpy as np
import scipy.sparse as sp
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn.pytorch import conv as dgl_conv
from startup_data_set import COMP4222Dataset
from PredictorClasses import *
from CustomMetrics import *
device = torch.device('cpu')

In [2]:
dataset = COMP4222Dataset()
graph = dataset[0]
print(graph)

Graph(num_nodes=25446, num_edges=45621,
      ndata_schemes={'feat': Scheme(shape=(221,), dtype=torch.float64), 'label': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'feat': Scheme(shape=(26,), dtype=torch.float64)})


In [3]:
in_feats = graph.ndata['feat'].shape[1]

In [4]:
u, v = graph.edges()
# give id for all edges then permutation
eids = np.arange(graph.number_of_edges())
eids = np.random.permutation(eids)

# use 10% as test set
test_size = int(len(eids) * 0.1)
train_size = graph.number_of_edges() - test_size

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing

#use sparse matrix to save memory
# ,shape = (torch.max(v)+1,torch.max(v)+1)
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(torch.max(u)+1,torch.max(v)+1)
neg_u, neg_v = np.where(adj_neg != 0) # negative edge, we don't have edge

neg_eids = np.random.choice(len(neg_u), graph.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [11]:
print(train_pos_u)

tensor([14974,  5534,  8323,  ..., 15352, 12288, 10852])


In [5]:
# Debug

In [6]:
train_g = dgl.remove_edges(graph, eids[:test_size])
train_g = dgl.add_self_loop(train_g)
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=graph.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=graph.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=graph.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=graph.number_of_nodes())

In [7]:
class GraphSAGEModel(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 out_dim,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGEModel, self).__init__()

        self.layers = nn.ModuleList()
        # input layer
        self.layers.append(dgl_conv.SAGEConv(in_feats, n_hidden, aggregator_type,
                                         feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(dgl_conv.SAGEConv(n_hidden, n_hidden, aggregator_type,
                                             feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(dgl_conv.SAGEConv(n_hidden, out_dim, aggregator_type,
                                         feat_drop=dropout, activation=None))
        
    def forward(self, g, features):
        h = features.float()
        for layer in self.layers:
            h = layer(g, h).float()
        return h

In [8]:
# Hyperparameters
n_hidden = 64
n_layers = 2
dropout = 0.5
aggregator_type = 'mean'

gconv_model = GraphSAGEModel(in_feats,
                             n_hidden,
                             n_hidden,
                             n_layers,
                             F.relu,
                             dropout,
                             aggregator_type)

In [9]:
pred = DotPredictor()

In [10]:
import itertools

optimizer = torch.optim.Adam(itertools.chain(gconv_model.parameters(), pred.parameters()), lr=0.01)
for e in range(200):
    # forward
    h = gconv_model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

In epoch 0, loss: 7.574316024780273
In epoch 5, loss: 0.8317697048187256
In epoch 10, loss: 0.6913649439811707
In epoch 15, loss: 0.6891352534294128
In epoch 20, loss: 0.6893130540847778
In epoch 25, loss: 0.6891641616821289
In epoch 30, loss: 0.6875033378601074
In epoch 35, loss: 0.6870720386505127
In epoch 40, loss: 0.6857577562332153
In epoch 45, loss: 0.6835238933563232
In epoch 50, loss: 0.6820350885391235
In epoch 55, loss: 0.6803224682807922
In epoch 60, loss: 0.677024781703949
In epoch 65, loss: 0.6754440665245056
In epoch 70, loss: 0.6719152927398682
In epoch 75, loss: 0.6640875339508057
In epoch 80, loss: 0.661032497882843
In epoch 85, loss: 0.6535555720329285
In epoch 90, loss: 0.6424782872200012
In epoch 95, loss: 0.6329498291015625
In epoch 100, loss: 0.6187270879745483
In epoch 105, loss: 0.6048311591148376
In epoch 110, loss: 0.5969589948654175
In epoch 115, loss: 0.5950630307197571
In epoch 120, loss: 0.5795830488204956
In epoch 125, loss: 0.5775408744812012
In epoch 13